In [ ]:
import cv2
import numpy as np
import cv2 as cv
import glob
import time
from tqdm import tqdm
from copy import deepcopy
import scipy.optimize as sopt
from klampt.math import se3,so3
from itertools import combinations
import pickle
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import open3d as o3d
import pandas as pd
from create_point_cloud import load_whole_point_cloud, load_point_cloud,get_masked_point_cloud
import scipy
import torch
# Copyright (c) OpenMMLab. All rights reserved.
from collections import deque
from queue import Queue
from threading import Event, Lock, Thread

import cv2

from mmpose.apis import (get_track_id, inference_top_down_pose_model,
                         init_pose_model, vis_pose_result)
from mmpose.core import apply_bugeye_effect, apply_sunglasses_effect
from mmpose.utils import StopWatch

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

try:
    import psutil
    psutil_proc = psutil.Process()
except (ImportError, ModuleNotFoundError):
    psutil_proc = None
from torch import nn
    
from utils import get_proper_image_paths, strip_dataset_parent_folder,get_aligned_dataset,find_hand_center

In [ ]:
MMPOSE_DIR = '/home/motion/Joao/classes/hri_kdc/final_project/mmpose'
det_config = '{}/demo/mmdetection_cfg/ssdlite_mobilenetv2_scratch_600e_coco.py'.format(MMPOSE_DIR)
det_checkpoint = 'https://download.openmmlab.com/mmdetection/v2.0/ssd/ssdlite_mobilenetv2_scratch_600e_coco/ssdlite_mobilenetv2_scratch_600e_coco_20210629_110627-974d9307.pth'
device = 'cuda:0'
enable_human_pose = 1
human_pose_config = '{}/configs/wholebody/2d_kpt_sview_rgb_img/topdown_heatmap/coco-wholebody/vipnas_res50_coco_wholebody_256x192_dark.py'.format(MMPOSE_DIR)
human_pose_checkpoint = 'https://download.openmmlab.com/mmpose/top_down/vipnas/vipnas_res50_wholebody_256x192_dark-67c0ce35_20211112.pth'
human_det_ids = [1]
buffer_size = 1
display_delay = 0
assert has_mmdet, 'Please install mmdet to run the demo.'
assert det_config is not None
assert det_checkpoint is not None

# build detection model
det_model = init_detector(
    det_config, det_checkpoint, device=device.lower())

# build pose models
pose_model_list = []
if enable_human_pose:
    pose_model = init_pose_model(
        human_pose_config,
        human_pose_checkpoint,
        device=device.lower())
    model_info = {
        'name': 'HumanPose',
        'model': pose_model,
        'cat_ids': human_det_ids,
        'bbox_color': (148, 139, 255),
    }
    pose_model_list.append(model_info)


# store pose history for pose tracking
pose_history_list = []

# for _ in range(len(pose_model_list)):
#     pose_history_list.append({'pose_results_last': [], 'next_id': 0})
    
# datasets_dir = '/home/motion/data/ECE598/our_dataset'
# scenes = sorted(glob(datasets_dir + '/*'))

# dataset_folder = scenes[0]

# clean_df =  get_aligned_dataset(dataset_folder,master_camera = 'cam_torso_depth')

In [ ]:
# frame = cv2.imread(clean_df.loc[0,'cam_torso_color'], cv2.IMREAD_COLOR)
# dataset_name = pose_model.cfg.data['test']['type']

# res,outputs = inference_top_down_pose_model(
#             pose_model,
#             frame,
#             format='xyxy',
#             dataset=dataset_name,return_heatmap = True)
# heatmap = outputs[0]['heatmap']

In [ ]:
# %%time
# points = get_masked_point_cloud(clean_df.loc[0,'cam_torso_depth'],'realsense_torso')
# points.flatten().shape

In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

In [ ]:
class Hands_Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, master_df,pose_model,history_length = 30,pred_horizon = 15):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
        """
        self.master_df = master_df
        self.history_length = history_length
        self.pred_horizon = pred_horizon
        self.max_idx = self.master_df.shape[0]
        mask = np.zeros(133)
        mask[:] = False
        mask[5:24] = True
        mask[91:] = True
        self.pose_model = pose_model
        self.mask = mask
        feature_lenght = 10800 + 187392
    def __len__(self):
        return self.master_df.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        imgs_idx = np.arange(idx,idx-self.history_length,-5)
        imgs_idx = np.clip(imgs_idx,0,self.max_idx)
        gt_idx = np.clip([idx+self.pred_horizon],0,self.max_idx)[0]
        features = []
        for img_idx in imgs_idx:
#             color = cv2.imread(self.master_df.loc[img_idx,'cam_torso_color'], cv2.IMREAD_COLOR)
            color = self.master_df.loc[img_idx,'pre_processed_torso_heatmap']
            point = get_masked_point_cloud(self.master_df.loc[img_idx,'cam_torso_depth'],'realsense_torso')
            feature = np.concatenate((point.flatten(),color.flatten()))
            features.append(feature)
#         colors = np.array(colors)
        features = np.array(features)
        gt_left = self.master_df.loc[gt_idx,'gt_left_hand']
        gt_right = self.master_df.loc[gt_idx,'gt_right_hand']
        gt = np.concatenate((gt_left,gt_right))
#         print(gt)
#         img_name = os.path.join(self.root_dir,
#                                 self.landmarks_frame.iloc[idx, 0])
#         image = io.imread(img_name)
#         landmarks = self.landmarks_frame.iloc[idx, 1:]
#         landmarks = np.array([landmarks])
#         landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'data': features, 'gt_pose': gt}


        return sample

In [ ]:
# raw_df = pd.read_pickle('./first_ground_truth_alice_no_sword1.pkl')
# clean_df = get_proper_image_paths(raw_df,'/home/motion/data/ECE598/our_dataset')
clean_df = pd.read_pickle('alice_no_sword_pre_processed.pkl')
dataset = Hands_Dataset(clean_df,pose_model)

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=16, shuffle=False,num_workers = 4,prefetch_factor= 8,persistent_workers=  True)

for i in train_dataloader:
    break

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers,pose_model):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
        self.pose_model = pose_model
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    def preprocess_input(self,x):
        res,outputs = inference_top_down_pose_model(
        self.pose_model,
        frame,
        format='xyxy',
        dataset=dataset_name,return_heatmap = True)
        heatmap = outputs[0]['heatmap'][0]
        heatmap = heatmap[self.mask,:,:].flatten()
    def extract_heatmap(self,img_file):
        
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [ ]:
data = []
mask = np.zeros(133).astype(bool)
mask[:] = False
mask[5:24] = True
mask[91:] = True
hm = []
for i in tqdm(clean_df.cam_torso_color):  
    try:
        inference_top_down_pose_model(
                pose_model,
                i,
                format='xyxy',
                dataset=dataset_name,return_heatmap = True)
        heatmap = outputs[0]['heatmap'][0]
        heatmap = heatmap[mask,:,:].flatten()
        hm.append(heatmap)
    except:
        hm.append(heatmap)

In [ ]:
clean_df['pre_processed_torso_heatmap'] = hm

In [ ]:
clean_df.to_pickle('alice_no_sword_pre_processed.pkl')

In [ ]:
clean_df.columns